should be answered using `n=10` period binomial model for the short-rate `r_{i,j}`. lattice param is `r_{0,0} = 5%`, `u=1.1`, `d=0.9`, and `q=1-q=1/2`

In [17]:
n = 10
u = 1.1
d = 0.9

## Helper Functions

In [4]:
def generate_short_rate_lattice(r00, n, u, d):
    short_rate = [[r00]]

    for i in range(n):
        # get the latest
        latest_rates = short_rate[-1]
        new_rates = []

        # go through each short rate and scale it up
        for j in range(len(latest_rates)):
            new_rate = u * latest_rates[j]
            new_rates.append(round(new_rate,5))

        # only need one scale down since it "propogates"
        last = d * latest_rates[-1]
        new_rates.append(round(last,5))

        # add this periods rates to the lattice
        short_rate.append(new_rates)

    return short_rate

# Q1

In [240]:
def generate_zcb_lattice(face_value, periods, short_rate_lattice, qu=0.5, qd=0.5, debug=False):
    # create values at maturity (face_value)
    lattice = [[face_value]*(periods + 1)]
    
    # step backward in time
    for i in range(periods):
        prev_prices = lattice[-1 - i]
        curr_short_rate = short_rate_lattice[periods - i  - 1]
        period_prices = []
        if debug: print(f"Period {periods - i  - 1}")
       
        # go through each state, there are x + 1 states in the xth period
        for state in range(periods - i):
            discount_rate = 1 / (1 + curr_short_rate[state])
            zij = discount_rate*(qu*prev_prices[state] + qd*prev_prices[state + 1])
            period_prices.append(zij)
            if debug: print(f"\tstate {state}\tshort rate: {curr_short_rate[state]}\tfirst pp: {prev_prices[state]}\tsecond pp: {prev_prices[state + 1]}")
        
        lattice.insert(0,period_prices)
        if debug: print(lattice) 

    return lattice

Price the ZCB that matures at `t=10`

In [241]:
short_rate_lattice = generate_short_rate_lattice(.05, 10, u=1.1, d=0.9)

In [242]:
zcb = generate_zcb_lattice(100, 10, short_rate_lattice)

In [243]:
q1 = zcb[0][0]

In [244]:
print(f"The ZCB can be fairly priced at ${round(q1, 2)}")

The ZCB can be fairly priced at $61.62


# Q2

In [245]:
def reduce_forward(forward_lattice, short_rate_lattice, periods, qu=.5, qd=.5, debug=True):
    
    # go through each time period
    for period in range(periods)[::-1]:
        new_prices = []
        prev_prices = forward_lattice[0]
        if debug: print(f"Period {period}\tlen curr price: {len(prev_prices) - 1}")
        if debug: print(f"Previous prices: {prev_prices}")

        # go through each state for the given time period
        for state in range(len(prev_prices) - 1):
            if debug: print(f"\t1st: {period}\t2nd: {state}")
                
            discount_rate = 1/(1 + short_rate_lattice[period][state])
            ex_discount_rnp = qu*prev_prices[state] + qd*prev_prices[state + 1]
            new_prices.append(discount_rate*ex_discount_rnp)

        forward_lattice.insert(0, new_prices)

    return forward_lattice

Price the forward on the ZCB matures at `t=4` (correct)

In [246]:
forward = reduce_forward([zcb[4]], short_rate_lattice, 4, debug=False)
q2 = forward[0][0]*100/generate_zcb_lattice(100, 4, short_rate_lattice)[0][0]
print(f"Fair value of the forward is: ${round(q2, 2)}")

Fair value of the forward is: $74.88


# Q3

In [247]:
def reduce_future(future_lattice, periods, qu=.5, qd=.5):
    
    for period in range(periods)[::-1]:
        prev_prices = future_lattice[0]
        new_prices = []
        for state in range(len(prev_prices) - 1):
            new_price = qu*prev_prices[state] + qd*prev_prices[state + 1]
            new_prices.append(new_price)

        future_lattice.insert(0, new_prices)

    return future_lattice

Price the future on the ZCB matures at `t=4` (correct)

In [248]:
zcb = generate_zcb_lattice(100, 10, short_rate_lattice)
future_prices = reduce_future([zcb[4]], 4, qu=.5, qd=.5)


In [249]:
q3 = future_prices[0][0]

In [250]:
print(f"The fair value of the future is: ${round(q3, 2)}")

The fair value of the future is: $74.82


# Q4

Price of american call option, option expires in `n=6` and `strike=80` (correct)

In [264]:
def reduce_call(values, underlying, short_rate_lattice, n=6, K=80, qu=.5, qd=.5, american=True, debug=False):
    for period in range(n)[::-1]:
        # get the necessary information for this period
        curr_underlying = underlying[period]
        curr_short_rates = short_rate_lattice[period]
        previous_values = values[0]
        
        new_results = []
        
        for state in range(period + 1):
            discount_rate = 1/(1 + curr_short_rates[state])
            risk_neutral_val = (qu*previous_values[state] + qd*previous_values[state + 1])*discount_rate
            
            if american:
                val = max(max(curr_underlying[state] - K, 0), risk_neutral_val)
            else:
                val = max(0, risk_neutral_val)

            new_results.append(val)
        
        values.insert(0, new_results)
    return values

In [272]:
n = 6
K = 80

underlying = zcb[6]
option_prices = []

# build all the final values for the option
for price in underlying:
    option_value = max(price - K, 0)
    option_prices.append(option_value)
    
option_prices = [option_prices]

# reduce the american call to the final value
american_call = reduce_call(option_prices, zcb, short_rate_lattice, n=6, K=80, american=True, debug=False)
q4 = american_call[0][0]
print(f"The fair value of the american call on the ZCB is ${round(q4, 2)}")

The fair value of the american call on the ZCB is $2.36


# Q5

Calc init value of forward-starting swap that begins at `t=1` with maturity `t=10` and a fixed rate of `4.5%`. Payments take plac ein arrears. Notional of 1,000,000.
correct answer in test is: 33374

In [308]:
def reduce_swap(fixed_rate, short_rate_lattice, start=1, periods=10, qu=.5, qd=.5, debug=True):
    # once start time is achieve, just reduce using RNP
    # at the very end, only coupon is calc'ed 
    # in the middle, both coupon and underlying (RNP) are combined

    def coupon(period, state):
        # base on some period and state, gets the coupon payment
        rate = short_rate_lattice[period][state]
        coupon = (rate - fixed_rate)/(1 + rate)

        return coupon
        
    if debug: print("started at the end")
    # build the final values that will be reduced
    swap_lattice = []
    for state in range(periods + 1):
        swap_lattice.append(coupon(periods, state))

    swap_lattice = [swap_lattice]
    if debug: print(swap_lattice[0])

    if debug: print("started middle")
    # reduce the swap intermediate part of the swap
    for period in range(start, periods)[::-1]:
        prev_coupon = swap_lattice[0] 
        new_coupon = []
    
        
        for state in range(period + 1):
            rate = short_rate_lattice[period][state]
            discount_rate = 1/(1 + rate)
            coupon = (rate - fixed_rate)
            
            rnp = (coupon + qu*prev_coupon[state] + qd*prev_coupon[state+1])*discount_rate
            new_coupon.append(rnp)
            
        swap_lattice.insert(0, new_coupon)
        if debug: print(swap_lattice[0])

   
    if debug: print("started the beginning part")
    # fill in the end if there's anything to do
    for i in range(start)[::-1]:
        prev_coupon = swap_lattice[0] 
        new_coupon = []

        for state in range(i + 1):
            discount_rate = 1/(1 + short_rate_lattice[i][state])
            rnp = (qu*prev_coupon[state] + qd*prev_coupon[state+1])*discount_rate

            new_coupon.append(rnp)

        swap_lattice.insert(0, new_coupon)
        if debug: print(swap_lattice[0])


    return swap_lattice

In [366]:
ps = 10
short_rate_lattice = generate_short_rate_lattice(.05, ps, u=1.1, d=0.9)
swap_lattice = reduce_swap(.045, short_rate_lattice, periods=ps, start=1, debug=False)
q5 = round(swap_lattice[0][0]*1_000_000,0)

0.05


In [360]:
print(f"The fair price on the forward-starting swap is ${q5}")

The fair price on the forward-starting swap is $33391.0


## Price using elementary pricing

Use forward equations to generate and elementary price lattice. Use this to value the same forward starting swap as above. 

In [361]:
def generate_elementary_prices(srl, periods=10):
    # srl is the short rate lattice
    P = [[1]]

    def p_k_s(k, s):
        # find some elementary security at time k+1 and state s
        # going to the right anywhere but bottom
        first = P[k-1][s-1]/(2*(1 + srl[k-1][s-1]))
        second = P[k-1][s]/(2*(1 + srl[k-1][s]))
        return first + second
    
    def p_k_0(k):
        # going to the right at the bottom
        num = P[k - 1][0]
        den = 2*(1+srl[k - 1][0])
        return num / den
    
    def p_k_k(k):
        # going diagonally at the top
        num = P[k-1][k-1]
        den = 2*(1 + srl[k-1][k-1])
        return num / den
    
    for k in range(1, periods + 1):
        new_period = []
        
        # do bottom
        new_period.append(p_k_0(k))

        # do i - 1 middle items
        for s in range(1, k):
            new_period.append(p_k_s(k, s))
        
        # do top
        new_period.append(p_k_k(k))
        
        P.append(new_period)
        
    return P

In [362]:
def forward_starting_swap(fixed_rate, short_rate_lattice, ep, start=1, periods=10, debug=False):
    # ep are elementary prices
    
    value = 0
    for period in range(start, periods + 1):
        if debug: print(f"period: {period}")
        for i in range(len(short_rate_lattice[period])):
            
            short_rate = short_rate_lattice[period][i]
            val = (short_rate - fixed_rate)/(1+short_rate)
            if debug: print(f"\t({fixed_rate} - {short_rate})/{(1+short_rate)} * {ep[period][i]}")
            value += val*ep[period][i]

            
    return value

In [365]:
pers = 10
short_rate_lattice = generate_short_rate_lattice(.05, pers, u=1.1, d=0.9)
elementary_prices = generate_elementary_prices(short_rate_lattice, periods=pers)
swap_val = forward_starting_swap(0.045, short_rate_lattice, elementary_prices, periods=pers, start=1, debug=False)
print(f"The initial value of the swap is ${round(swap_val*1000000)}")

The initial value of the swap is $33391


# Q6

Compute the initial price of a swaption with strike `0` and that matures at `t=5`. Once exercised, all future cash flows of the swap are received. Same swap as mentioned above.

In [356]:
values = [[max(x, 0)for x in swap_lattice[5]]]
call = reduce_call(values, swap_lattice, short_rate_lattice, n=5, K=0, qu=.5, qd=.5, american=False, debug=False)

In [357]:
actual_q6 = 26311
q6 = round(call[0][0]*1_000_000)
print(f"Value of the swaption is ${q6}")

Value of the swaptions is $26315


In [358]:
print(f"Inaccuracy of answer for Q5: {round((1-33374/q5)*100,3)}%")
print(f"Inaccuracy of answer for Q6: {round((1-actual_q6/q6)*100,3)}%")

Inaccuracy of answer for Q5: 0.051%
Inaccuracy of answer for Q6: 0.015%
